In [4]:
from dotenv import load_dotenv
load_dotenv()

import os
import time
import requests

BASE_URL = os.environ["OLLAMA_BASE_URL"]
MODEL_A = os.environ["OLLAMA_MODEL_A"]
MODEL_B = os.environ["OLLAMA_MODEL_B"]


In [ ]:
def call_ollama(model, prompt):
    print(f"Calling {model}...", flush=True)

    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "num_predict": 40,               # HARD LIMIT
            "stop": ["\n\n", "Answer:"]      # HARD STOP
        }
    }

    r = requests.post(
        f"{BASE_URL}/api/generate",
        json=payload,
        timeout=20
    )

    data = r.json()
    if "response" not in data:
        raise RuntimeError(data)

    return data["response"].strip()


In [6]:
def debate(topic):
    print(f"\n🔥 DEBATE STARTED: {topic}\n")

    history = []
    max_turns = 6   # HARD CAP (prevents infinite runs)

    for turn in range(max_turns):
        side = "FOR" if turn % 2 == 0 else "AGAINST"
        model = MODEL_A if side == "FOR" else MODEL_B

        context = "\n".join(history[-1:])  # VERY SMALL CONTEXT

        prompt = f"""
Topic: {topic}

Give ONE short argument {side}.
Answer in 1–2 sentences max.

Previous:
{context}

Answer:
"""

        try:
            reply = call_ollama(model, prompt)
        except Exception as e:
            reply = f"[ERROR: {e}]"

        history.append(reply)

        print(f"{side}:\n{reply}\n{'-'*40}")
        time.sleep(2)

    print("\n🛑 DEBATE ENDED\n")


In [7]:
debate("Should governments heavily regulate artificial intelligence?")



🔥 DEBATE STARTED: Should governments heavily regulate artificial intelligence?

⏳ Calling gemma3:270m...
FOR:
Generally, governments should heavily regulate artificial intelligence (AI) to ensure its development and deployment is aligned with societal values and ethical considerations. This prevents harmful applications, promotes fairness, and protects individuals from potential
----------------------------------------
⏳ Calling gemma3:270m...
AGAINST:
Generally, governments should heavily regulate artificial intelligence (AI) to ensure its development and deployment is aligned with societal values and ethical considerations. This prevents harmful applications, promotes fairness, and protects individuals from potential
----------------------------------------
⏳ Calling gemma3:270m...
FOR:
Regulating AI is crucial for ensuring its development and deployment are aligned with societal values and ethical considerations, mitigating risks such as bias, job displacement, and misuse.
--------